# ENEXIS Graduation Project

#### Upload of data from CBS datasets "Zonenstrom wijken en buurten" of 2016- 2019

In [1]:
import cbsodata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nbconvert
pd.set_option("max_rows", 120)
pd.options.mode.chained_assignment = None  # default='warn'

Lets select a data set from CBS. 

In [2]:
# https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=85010NED&_theme=278

datasets = ('85010NED', #Zonnestroom; wijken en buurten, 2019   # 0
            '84772NED', #Zonnestroom; wijken en buurten, 2018   # 1
            '84517NED', #Zonnestroom; wijken en buurten, 2017   # 2
            '84131NED') #Zonnestroom; wijken en buurten, 2016   # 3

In [3]:
data_years = {'85010NED' : 2019, 
              '84772NED' : 2018,
              '84517NED' : 2017,
              '84131NED' : 2016}

In [20]:
df = pd.DataFrame()
for i in range(len(datasets)):
    df_new = pd.DataFrame(cbsodata.get_data(datasets[i]))
    df_new['Year'] = data_years[datasets[i]]
    df = pd.concat([df, df_new], axis = 0)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66759 entries, 0 to 16196
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ID                                   66759 non-null  int64  
 1   WijkenEnBuurten                      66759 non-null  object 
 2   Gemeentenaam_1                       66759 non-null  object 
 3   SoortRegio_2                         66759 non-null  object 
 4   Codering_3                           66759 non-null  object 
 5   IndelingswijzigingWijkenEnBuurten_4  66759 non-null  object 
 6   AantalInstallatiesBijWoningen_5      53994 non-null  float64
 7   OpgesteldVermogenVanZonnepanelen_6   53987 non-null  float64
 8   Year                                 66759 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 5.1+ MB


In [22]:
df.shape

(66759, 9)

In [23]:
df_nans = df[df['OpgesteldVermogenVanZonnepanelen_6'].isnull()] # where are the missing values for the PV installations?

In [24]:
df_nans['Year'].value_counts() # the number seems to decrease over time. Hypothesis: the nans are in fact 0 values

2016    3826
2017    3344
2018    2921
2019    2681
Name: Year, dtype: int64

In [25]:
df_nans['SoortRegio_2'].value_counts()

Buurt         11892
Wijk            795
Gemeente         84
                  1
Name: SoortRegio_2, dtype: int64

In [26]:
df_nans['Gemeentenaam_1'].value_counts()

Amsterdam                                   947
Lelystad                                    375
Tilburg                                     299
Dordrecht                                   201
Zwijndrecht                                 182
                                           ... 
Slochteren                                    2
Reusel-De Mierden                             2
                                              1
Borne                                         1
Westerkwartier                                1
Name: Gemeentenaam_1, Length: 365, dtype: int64

In [27]:
df[df['OpgesteldVermogenVanZonnepanelen_6'] == 0] # there are no 0 values, so the nans are most likely 0

,ID,WijkenEnBuurten,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingWijkenEnBuurten_4,AantalInstallatiesBijWoningen_5,OpgesteldVermogenVanZonnepanelen_6,Year


In [28]:
df[df['AantalInstallatiesBijWoningen_5'] == 0]

,ID,WijkenEnBuurten,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingWijkenEnBuurten_4,AantalInstallatiesBijWoningen_5,OpgesteldVermogenVanZonnepanelen_6,Year


We shall replace all nans with 0

In [29]:
df = df.fillna(0)

In [34]:
df['SoortRegio_2'] = df['SoortRegio_2'].str.strip()

In [37]:
df = df[df['SoortRegio_2'] == 'Buurt']

In [39]:
df = df.drop(columns = ['ID', 'WijkenEnBuurten', 'Gemeentenaam_1', 'SoortRegio_2'], axis = 1)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52933 entries, 3 to 16196
Data columns (total 5 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Codering_3                           52933 non-null  object 
 1   IndelingswijzigingWijkenEnBuurten_4  52933 non-null  object 
 2   AantalInstallatiesBijWoningen_5      52933 non-null  float64
 3   OpgesteldVermogenVanZonnepanelen_6   52933 non-null  float64
 4   Year                                 52933 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 2.4+ MB


In [41]:
df.head()

,Codering_3,IndelingswijzigingWijkenEnBuurten_4,AantalInstallatiesBijWoningen_5,OpgesteldVermogenVanZonnepanelen_6,Year
3,BU16800000,1,396.0,1426.0,2019
4,BU16800009,1,12.0,50.0,2019
6,BU16800100,1,132.0,489.0,2019
7,BU16800109,1,12.0,57.0,2019
9,BU16800200,1,32.0,142.0,2019


In [42]:
df['IndelingswijzigingWijkenEnBuurten_4'].value_counts()

1    47878
3     3897
2     1158
Name: IndelingswijzigingWijkenEnBuurten_4, dtype: int64

In [44]:
c_path = "./data/"
v_file = "CSB Buurten Indeling" 

In [64]:
df1     = pd.read_csv(filepath_or_buffer = c_path + v_file + ".csv",
                      encoding           = 'UTF-8')

In [65]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14174 entries, 0 to 14173
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BU_2021          14174 non-null  object
 1   BU_NAAM          14174 non-null  object
 2   WK_2021          14174 non-null  object
 3   WK_NAAM          14174 non-null  object
 4   GM_2021          14174 non-null  object
 5   GM_NAAM          14174 non-null  object
 6   ProvinciecodePV  14174 non-null  object
 7   Provincienaam    14174 non-null  object
dtypes: object(8)
memory usage: 996.6+ KB


In [66]:
df1.head()

,BU_2021,BU_NAAM,WK_2021,WK_NAAM,GM_2021,GM_NAAM,ProvinciecodePV,Provincienaam
0,BU00140000,Binnenstad-Noord,WK001400,Centrum,GM0014,Groningen,PV20,Groningen
1,BU00140001,Binnenstad-Zuid,WK001400,Centrum,GM0014,Groningen,PV20,Groningen
2,BU00140002,Binnenstad-Oost,WK001400,Centrum,GM0014,Groningen,PV20,Groningen
3,BU00140003,Binnenstad-West,WK001400,Centrum,GM0014,Groningen,PV20,Groningen
4,BU00140004,Noorderplantsoen,WK001400,Centrum,GM0014,Groningen,PV20,Groningen


In [68]:
df = df.merge(df1, left_on= 'Codering_3', right_on = 'BU_2021')

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46192 entries, 0 to 46191
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Codering_3                           46192 non-null  object 
 1   IndelingswijzigingWijkenEnBuurten_4  46192 non-null  object 
 2   AantalInstallatiesBijWoningen_5      46192 non-null  float64
 3   OpgesteldVermogenVanZonnepanelen_6   46192 non-null  float64
 4   Year                                 46192 non-null  int64  
 5   BU_2021                              46192 non-null  object 
 6   BU_NAAM                              46192 non-null  object 
 7   WK_2021                              46192 non-null  object 
 8   WK_NAAM                              46192 non-null  object 
 9   GM_2021                              46192 non-null  object 
 10  GM_NAAM                              46192 non-null  object 
 11  ProvinciecodePV             

In [72]:
df = df.drop(columns = 'Codering_3', axis = 1)

In [74]:
df = df[['Year', 'BU_2021', 'BU_NAAM', 'WK_2021', 'WK_NAAM', 'GM_2021',
       'GM_NAAM', 'ProvinciecodePV', 'Provincienaam', 'IndelingswijzigingWijkenEnBuurten_4',
       'AantalInstallatiesBijWoningen_5', 'OpgesteldVermogenVanZonnepanelen_6']]

In [89]:
df['Year'] = pd.to_datetime(df.Year, format='%Y')

In [90]:
df.head()

,Year,BU_2021,BU_NAAM,WK_2021,WK_NAAM,GM_2021,GM_NAAM,ProvinciecodePV,Provincienaam,IndelingswijzigingWijkenEnBuurten_4,AantalInstallatiesBijWoningen_5,OpgesteldVermogenVanZonnepanelen_6
0,2019-01-01,BU16800000,Annen,WK168000,Wijk 00 Annen,GM1680,Aa en Hunze,PV22,Drenthe,1,396.0,1426.0
1,2018-01-01,BU16800000,Annen,WK168000,Wijk 00 Annen,GM1680,Aa en Hunze,PV22,Drenthe,1,308.0,1068.0
2,2017-01-01,BU16800000,Annen,WK168000,Wijk 00 Annen,GM1680,Aa en Hunze,PV22,Drenthe,1,191.0,726.0
3,2016-01-01,BU16800000,Annen,WK168000,Wijk 00 Annen,GM1680,Aa en Hunze,PV22,Drenthe,1,128.0,504.0
4,2019-01-01,BU16800009,Verspreide huizen Annen,WK168000,Wijk 00 Annen,GM1680,Aa en Hunze,PV22,Drenthe,1,12.0,50.0
